In [1]:
#IMPORTANDO BIBLIOTECAS
import requests
import json
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
import datetime
import os

In [2]:
#GUARDANDO LISTAS
Urls_Magalu = []
Sellers_Magalu = []
Country_Magalu = []
Price_Magalu = []
SKU_Magalu = []
Title_Magalu = []
Installment_Magalu_quantidade = []
Installment_Magalu_valor_parcela = []
Installment_Magalu_valor_total = []

In [8]:
def getting_n_creating_magazine_urls(brand):
    # Pegando caminho do database
    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "%2B")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "%2B")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.magazineluiza.com.br/_next/data/DY3NRy49CNguQmhPOTqj2/busca/" + \
    df['Brand'][0] + "%2B" + df['Product_Name'] + ".json?slug=busca&slug=" + \
    df['Brand'][0] + "%2B" + df['Product_Name']

    return df

In [9]:
Df_products_raw = getting_n_creating_magazine_urls("Wacom")
Df_products_raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom+ctl472,https://www.magazineluiza.com.br/_next/data/DY...
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom+ctl4100,https://www.magazineluiza.com.br/_next/data/DY...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wle0,https://www.magazineluiza.com.br/_next/data/DY...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wlk0,https://www.magazineluiza.com.br/_next/data/DY...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wlk0,https://www.magazineluiza.com.br/_next/data/DY...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom+dtc133w0a,https://www.magazineluiza.com.br/_next/data/DY...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom+pth460,https://www.magazineluiza.com.br/_next/data/DY...
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom+dtk1660k0a,https://www.magazineluiza.com.br/_next/data/DY...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom+pth660,https://www.magazineluiza.com.br/_next/data/DY...
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wle0,https://www.magazineluiza.com.br/_next/data/DY...


In [12]:
headers_magalu = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.59"}

In [13]:
def get_attributes(url):
    time.sleep(20)

    response = requests.get(url, headers=headers_magalu)
    text_json = response.json()

    # URL
    id_url = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Urls_Magalu.append(text_json['pageProps']['data']['search']['products'][id_url]['url'])
        except:
            Urls_Magalu.append("ERRO")

        id_url = id_url + 1

    # ID
    id_number = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            SKU_Magalu.append(text_json['pageProps']['data']['search']['products'][id_number]['variationId'])
        except:
            SKU_Magalu.append("ERRO")

        id_number = id_number + 1

    # SELLER
    id_seller = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Sellers_Magalu.append(
                text_json['pageProps']['data']['search']['products'][id_seller]['seller']['description'])
        except:
            Sellers_Magalu.append("ERRO")

        id_seller = id_seller + 1

    # COUNTRY
    id_country = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Country_Magalu.append(
                text_json['pageProps']['data']['search']['products'][id_country]['seller']['details']['address'][
                    'country'])
        except:
            Country_Magalu.append("ERRO")

        id_country = id_country + 1

    # PREÇO
    id_price = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Price_Magalu.append(text_json['pageProps']['data']['search']['products'][id_price]['price']['price'])
        except:
            Price_Magalu.append("ERRO")

        id_price = id_price + 1

    # TITULO
    id_title = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Title_Magalu.append(text_json['pageProps']['data']['search']['products'][id_title]['title'])
        except:
            Title_Magalu.append("ERRO")

        id_title = id_title + 1

    # INSTALLMENT QUANTIDADE
    id_installment_quantidade = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_quantidade.append(
                text_json['pageProps']['data']['search']['products'][id_installment_quantidade]['installment'][
                    'quantity'])
        except:
            Installment_Magalu_quantidade.append("ERRO")

        id_installment_quantidade = id_installment_quantidade + 1

    # INSTALLMENT VALOR POR PARCELA
    id_installment_valor_parcela = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_valor_parcela.append(
                text_json['pageProps']['data']['search']['products'][id_installment_valor_parcela]['installment'][
                    'amount'])
        except:
            Installment_Magalu_valor_parcela.append("ERRO")

        id_installment_valor_parcela = id_installment_valor_parcela + 1

    # INSTALLMENT VALOR TOTAL
    id_installment_valor_total = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_valor_total.append(
                text_json['pageProps']['data']['search']['products'][id_installment_valor_total]['installment'][
                    'totalAmount'])
        except:
            Installment_Magalu_valor_total.append("ERRO")

        id_installment_valor_total = id_installment_valor_total + 1

In [14]:
for url in tqdm(Df_products_raw['Urls_search']):
    get_attributes(url)

100%|██████████| 10/10 [03:31<00:00, 21.13s/it]


In [15]:
def creating_dataframe(urls, sellers, price, sku, title, quantidade, parcela, total, brand):
    Dataframe = pd.DataFrame()

    Hoje = pd.to_datetime('today', errors='ignore').date()

    Dataframe['Urls_2'] = urls

    Dataframe['DATE'] = Hoje

    Dataframe['URL'] = "https://www.magazineluiza.com.br/" + Dataframe['Urls_2']
    Dataframe['MARKETPLACE'] = 'MAGAZINE LUIZA'
    Dataframe['SELLER'] = sellers

    Dataframe['PRICE'] = price
    Dataframe['PRICE'] = Dataframe['PRICE'].astype('float')


    Dataframe['INSTALLMENT'] = quantidade
    Dataframe['PARCEL'] = parcela

    Dataframe['INSTALLMENT_PAYMENT'] = total

    Dataframe['ID'] = sku
    Dataframe['PRODUCT'] = title

    if brand == "GoPro":
        Dataframe = Dataframe[Dataframe['PRICE'] > 1000]
    elif brand == 'Motorola':
        Dataframe = Dataframe[Dataframe['PRICE'] > 100]
    elif brand == 'Wacom':
        Dataframe = Dataframe[Dataframe['PRICE'] > 100]

    #Colocando na ordem correta
    Dataframe = Dataframe[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT']]


    return Dataframe

In [16]:
dataset_magalu = creating_dataframe(Urls_Magalu,Sellers_Magalu,Price_Magalu,SKU_Magalu,Title_Magalu,Installment_Magalu_quantidade,Installment_Magalu_valor_parcela,Installment_Magalu_valor_total,'Wacom')


In [17]:
dataset_magalu

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT
0,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Magalu,599.00,123.33,3,369.99,218538400,Mesa Digitalizadora Wacom Pequena
1,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,SHOPAR,287.02,54.18,5,270.90,cdh7fb5df7,Mesa Digitalizadora Wacom One CTL-472 Pequena ...
2,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,KaBuM!,235.18,58.80,4,235.18,jeff7hbg33,Mesa Digitalizadora One by Wacom CTL472 Pequena
3,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Miranda Computação,314.00,62.80,5,314.00,bb7f1kfbj8,Mesa Digitalizadora One by Wacom CTL472 WACOM
4,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Primetek,329.90,47.13,7,329.90,cg2ck1b5hk,Mesa Digitalizadora One by Wacom CTL472 Pequena
...,...,...,...,...,...,...,...,...,...,...
136,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,e-spot,1909.90,190.99,10,1909.90,khg7205477,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...
137,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Inpower,1899.00,152.80,10,1528.00,eah018g7h7,Mesa Digitalizadora Wacom Intuos M Bluetooth P...
138,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,W-Tech Imports,2297.00,199.70,10,1997.00,hc31e0gkf5,Mesa Digitalizadora Wacom Intuos CTL6100WLE0 B...
139,2022-05-20,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Miranda Computação,2014.00,201.40,10,2014.00,ha211gfj8c,"Mesa Digitalizadora Intuos Bluetooth, Média, V..."


In [19]:
current = os.getcwd()
current

'c:\\Users\\pedro\\Documents\\Turte Brand Protection\\Turtle_Thinker_Alpha_0.1\\Testes_Spiders'

In [20]:
path = current + '\Teste_Magalu.xlsx'

In [21]:
path

'c:\\Users\\pedro\\Documents\\Turte Brand Protection\\Turtle_Thinker_Alpha_0.1\\Testes_Spiders\\Teste_Magalu.xlsx'

In [22]:
dataset_magalu.to_excel(path, index=False)

In [16]:
dataset_magalu.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                  object
INSTALLMENT             object
INSTALLMENT_PAYMENT     object
ID                      object
PRODUCT                 object
dtype: object